In [1]:
import os
import sys
import numpy as np
import pandas as pd

import rocks

rocks.set_log_level("error")

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [2]:
import figure_setup as fs

# Get ZTF fit

In [3]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")
damit_file = os.path.join(data_fink, "data", "damit_abc.csv")

In [4]:
# Threshold for selection (of non-zero values)
thres = 1e-3

# Minimum phase angle to consider
min_phase = 3

In [5]:
# ZTF filters 1: g, 2: r
filters = {"1": "g", "2": "r"}

fink_colors = ["#15284F", "#F5622E"]

color_fink = "#15284F"

In [58]:
damit = pd.read_csv(damit_file)
# damit.head(5)

In [59]:
damit

,model_id,asteroid_id,lambda,beta,period,number,name,designation,reference_id,bibcode,a,b,c,R
0,101,101,35.0,-12.0,7.813230,2.0,Pallas,NaN,106,2003icar..164..346t,2.600240e+06,2.464155e+06,2.282461e+06,0.902027
1,101,101,35.0,-12.0,7.813230,2.0,Pallas,NaN,139,2011icar..214..652d,2.600240e+06,2.464155e+06,2.282461e+06,0.902027
2,106,104,340.0,42.0,7.274471,6.0,Hebe,NaN,106,2003icar..164..346t,1.690630e+05,1.634407e+05,1.447296e+05,0.870793
3,106,104,340.0,42.0,7.274471,6.0,Hebe,NaN,139,2011icar..214..652d,1.690630e+05,1.634407e+05,1.447296e+05,0.870793
4,110,106,335.0,-5.0,12.866670,8.0,Flora,NaN,106,2003icar..164..346t,8.951272e+04,8.496190e+04,7.833080e+04,0.898516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16312,16275,10853,326.0,-83.0,8.016800,99949.0,Miepgies,NaN,665,2023arxiv230510798d,8.597170e-01,5.586180e-01,5.387130e-01,0.795492
16313,16276,10854,34.0,-42.0,143.500000,99973.0,NaN,NaN,665,2023arxiv230510798d,9.555900e-01,5.506200e-01,4.848120e-01,0.693913
16314,16277,10854,194.0,-64.0,143.490000,99973.0,NaN,NaN,665,2023arxiv230510798d,1.003388e+00,5.521440e-01,4.607300e-01,0.646806
16315,16278,10855,192.0,-52.0,7.071400,99990.0,NaN,NaN,665,2023arxiv230510798d,1.090510e+00,5.518300e-01,4.180810e-01,0.570504


In [7]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

In [8]:
data.columns

Index(['ssnamenr', 'HG_chi2red', 'HG_status', 'HG_fit', 'HG_rms', 'HG_rms_g',
       'HG_rms_r', 'HG_median_error_phot', 'HG_median_error_phot_1',
       'HG_median_error_phot_2', 'HG_H_g', 'HG_dH_g', 'HG_G_g', 'HG_dG_g',
       'HG_H_r', 'HG_dH_r', 'HG_G_r', 'HG_dG_r', 'HG_flag', 'HG1G2_chi2red',
       'HG1G2_status', 'HG1G2_fit', 'HG1G2_rms', 'HG1G2_rms_g', 'HG1G2_rms_r',
       'HG1G2_median_error_phot', 'HG1G2_median_error_phot_1',
       'HG1G2_median_error_phot_2', 'HG1G2_H_g', 'HG1G2_dH_g', 'HG1G2_G1_g',
       'HG1G2_dG1_g', 'HG1G2_G2_g', 'HG1G2_dG2_g', 'HG1G2_H_r', 'HG1G2_dH_r',
       'HG1G2_G1_r', 'HG1G2_dG1_r', 'HG1G2_G2_r', 'HG1G2_dG2_r', 'HG1G2_flag',
       'SHG1G2_chi2red', 'SHG1G2_min_cos_lambda', 'SHG1G2_mean_cos_lambda',
       'SHG1G2_max_cos_lambda', 'SHG1G2_status', 'SHG1G2_fit', 'SHG1G2_rms',
       'SHG1G2_rms_g', 'SHG1G2_rms_r', 'SHG1G2_median_error_phot',
       'SHG1G2_median_error_phot_1', 'SHG1G2_median_error_phot_2', 'n_obs',
       'n_obs_g', 'n_obs_r', 

In [9]:
data["SHG1G2_dSpin"] = np.sqrt(
    (data["SHG1G2_dalpha0"] * np.cos(np.radians(data["SHG1G2_delta0"]))) ** 2
    + data["SHG1G2_ddelta0"] ** 2
)


# # Remove solutions above 90 deg of latitude
cond = data.SHG1G2_delta0 > 90
data.loc[cond, "SHG1G2_delta0"] = 90
print(f"above 90: {len(data[cond])} ")

cond = data.SHG1G2_delta0 < -90
data.loc[cond, "SHG1G2_delta0"] = -90
print(f"below 90: {len(data[cond])} ")

above 90: 0 
below 90: 0 


In [10]:
thres = 1e-3

# HG Parameeter
mask_HG_g = (data.HG_H_g.notna()) & (data.HG_G_g.notna())
mask_HG_r = (data.HG_H_r.notna()) & (data.HG_G_r.notna())
mask_HG_fit = (data.HG_fit == 0) & (data.HG_status >= 2)
mask_HG = mask_HG_g & mask_HG_r & mask_HG_fit

# HG1G2 parameters
mask_HG1G2_g = (
    (data.HG1G2_G1_g > thres)
    & (data.HG1G2_G2_g > thres)
    & ((1 - data.HG1G2_G1_g - data.HG1G2_G2_g) > thres)
)
mask_HG1G2_r = (
    (data.HG1G2_G1_r > thres)
    & (data.HG1G2_G2_r > thres)
    & ((1 - data.HG1G2_G1_r - data.HG1G2_G2_r) > thres)
)
mask_HG1G2_fit = (data.HG1G2_fit == 0) & (data.HG1G2_status >= 2)
mask_HG1G2 = mask_HG1G2_fit & mask_HG1G2_g & mask_HG1G2_r

# SHG1G2 ZTF
mask_SHG1G2_g = (
    (data.SHG1G2_G1_g > thres)
    & (data.SHG1G2_G2_g > thres)
    & ((1 - data.SHG1G2_G1_g - data.SHG1G2_G2_g) > thres)
)
mask_SHG1G2_r = (
    (data.SHG1G2_G1_r > thres)
    & (data.SHG1G2_G2_r > thres)
    & ((1 - data.SHG1G2_G1_r - data.SHG1G2_G2_r) > thres)
)
mask_SHG1G2_ZTF = mask_SHG1G2_g & mask_SHG1G2_r
mask_SHG1G2_fit = (data.SHG1G2_fit == 0) & (data.SHG1G2_status >= 2)
mask_SHG1G2 = mask_SHG1G2_fit & mask_SHG1G2_ZTF

# Oblateness
mask_R = data.SHG1G2_R > 0.3

# Spin solution suspicous: RA=={0,180,360}, DEC==0
maskSpin = (
    (data.SHG1G2_alpha0 > thres)
    & (np.abs(360 - data.SHG1G2_alpha0) > thres)
    & (np.abs(data.SHG1G2_alpha0 - 180) > thres)
    & (np.abs(data.SHG1G2_delta0) > thres)
)

# FINK Sample
maskFINK = mask_SHG1G2 & mask_R & maskSpin

# Phase coverage
maskPhase = data.min_phase < 2.5

# Global mask
mask = mask_HG1G2 & mask_SHG1G2


print(f" All data       : {len(data):6d}  ({100:>6.2f}%)")
print()
print(
    f"  Mask HG g      : {len(data[mask_HG_g]):6d}  ({100.*len(data[mask_HG_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG r      : {len(data[mask_HG_r]):6d}  ({100.*len(data[mask_HG_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG g+r    : {len(data[mask_HG]):6d}  ({100.*len(data[mask_HG])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask HG1G2 g   : {len(data[mask_HG1G2_g]):6d}  ({100.*len(data[mask_HG1G2_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG1G2 r   : {len(data[mask_HG1G2_r]):6d}  ({100.*len(data[mask_HG1G2_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask HG1G2 g+r : {len(data[mask_HG1G2]):6d}  ({100.*len(data[mask_HG1G2])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask SHG1G2 g  : {len(data[mask_SHG1G2_g]):6d}  ({100.*len(data[mask_SHG1G2_g])/len(data):>6.2f}%)"
)
print(
    f"  Mask SHG1G2 r  : {len(data[mask_SHG1G2_r]):6d}  ({100.*len(data[mask_SHG1G2_r])/len(data):>6.2f}%)"
)
print(
    f"  Mask SHG1G2 g+r: {len(data[mask_SHG1G2]):6d}  ({100.*len(data[mask_SHG1G2])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask Oblateness: {len(data[mask_R]):6d}  ({100.*len(data[mask_R])/len(data):>6.2f}%)"
)
print(
    f"  Mask Spin      : {len(data[maskSpin]):6d}  ({100.*len(data[maskSpin])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask FINK      : {len(data[maskFINK]):6d}  ({100.*len(data[maskFINK])/len(data):>6.2f}%)"
)
print()
print(
    f"  Mask (both)    : {len(data[mask]):6d}  ({100.*len(data[mask])/len(data):>6.2f}%)"
)
for minphase in [2, 3, 4, 5]:
    maskPhase = data.min_phase < minphase
    print(
        f"  Mask phase {minphase}  : {len(data[maskPhase]):6d}  ({100.*len(data[maskPhase])/len(data):>6.2f}%)"
    )


# len(data), len(data[mask]), len(data[mask_HG1G2]), len(data[mask_HG1G2sp]), len(data[mask_SHG1G2_g]), len(data[mask_SHG1G2_r]), len(data[mask_SHG1G2])

 All data       : 122675  (100.00%)

  Mask HG g      : 122371  ( 99.75%)
  Mask HG r      : 122373  ( 99.75%)
  Mask HG g+r    : 120010  ( 97.83%)

  Mask HG1G2 g   :  63922  ( 52.11%)
  Mask HG1G2 r   :  68956  ( 56.21%)
  Mask HG1G2 g+r :  47175  ( 38.46%)

  Mask SHG1G2 g  :  79912  ( 65.14%)
  Mask SHG1G2 r  :  84779  ( 69.11%)
  Mask SHG1G2 g+r:  64209  ( 52.34%)

  Mask Oblateness:  91745  ( 74.79%)
  Mask Spin      : 118811  ( 96.85%)

  Mask FINK      :  50163  ( 40.89%)

  Mask (both)    :  40935  ( 33.37%)
  Mask phase 2  :  62923  ( 51.29%)
  Mask phase 3  :  86161  ( 70.24%)
  Mask phase 4  : 100699  ( 82.09%)
  Mask phase 5  : 109011  ( 88.86%)


# Histogram of oblateness R

In [11]:
# --------------------------------------------------------------------------------
fig, ax = plt.subplots(
    figsize=fs.figsize(0.5),
    gridspec_kw={
        "hspace": 0.02,
        "wspace": 0.02,
        "top": 0.98,
        "bottom": 0.13,
        "left": 0.10,
        "right": 0.98,
    },
)

# --------------------------------------------------------------------------------
# FINK R
r = [0.3, 1]
b = 50


ax.hist(
    damit["R"],
    range=r,
    bins=b,
    density=True,
    label=f"DAMIT ({len(damit):,d})",
    color="slategray",
    alpha=0.8,
    rasterized=True,
)


ax.hist(
    data.loc[maskFINK, "SHG1G2_R"],
    range=r,
    bins=b,
    density=True,
    label=f"FINK ({len(data[maskFINK]):,d})",
    color=color_fink,
    alpha=0.8,
    rasterized=True,
)


# --------------------------------------------------------------------------------
# Axes
ax.set_xlabel("Oblateness R")
ax.set_ylabel("Density count")

ax.set_ylim(0, 5)
ax.legend(loc="upper left")

# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "distrib_R.png"),
    facecolor="white",
)

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "distrib_R.pgf"),
)
plt.close()

In [12]:
print(f"\\newcommand{{\damitSSO}}{{{damit['asteroid_id'].nunique():,d}\\xspace}}")
print(f"\\newcommand{{\damitModel}}{{{len(damit):,d}\\xspace}}")

\newcommand{\damitSSO}{10,743\xspace}
\newcommand{\damitModel}{16,317\xspace}


<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_281346/1699003260.py:1: SyntaxWarning: invalid escape sequence '\d'
  print( f"\\newcommand{{\damitSSO}}{{{damit['asteroid_id'].nunique():,d}\\xspace}}")
/tmp/ipykernel_281346/1699003260.py:2: SyntaxWarning: invalid escape sequence '\d'
  print( f"\\newcommand{{\damitModel}}{{{len(damit):,d}\\xspace}}")


below is exploratory

# Check RMS of SSOs with shape models

In [13]:
nn = rocks.id(data.ssnamenr)
data["sso_name"] = [n[0] for n in nn]

ERROR    [rocks] Could not identify '2018 LM04'.

ERROR    [rocks] Could not identify '2015 WV01'.

ERROR    [rocks] Could not identify '2001 YU03'.

ERROR    [rocks] Could not identify '2020 AV02'.

ERROR    [rocks] Could not identify '2003 YM01'.

...

ERROR    [rocks] Could not identify '2011 LZ05'.

ERROR    [rocks] Could not identify '2021 NX07'.

ERROR    [rocks] Could not identify '2003 RK07'.

ERROR    [rocks] Could not identify '2016 OF02'.

ERROR    [rocks] Could not identify '2020 JQ00'.

In [14]:
nn = rocks.id(damit.number)
damit["sso_name"] = [n[0] for n in nn]

In [15]:
fig, ax = plt.subplots(
    figsize=fs.figsize(0.5), gridspec_kw={"right": 0.975, "top": 0.95, "bottom": 0.15}
)

# Plot FINK Sample
r = [0, 0.5]
b = 50

# all = data[maskFINK].merge(damit, on="sso_name", how="left")


# Histogram FINK
ax.hist(
    data.loc[maskFINK, "SHG1G2_rms"],
    range=r,
    bins=b,
    color=color_fink,
    density=True,
    alpha=0.8,
    label=f"FINK ({len(data[maskFINK]):,d})",
)


# Histogram 3dShape
cond = maskFINK & data.sso_name.isin(damit.sso_name)
ax.hist(
    data.loc[cond, "SHG1G2_rms"],
    range=r,
    bins=b,
    color="slategray",
    density=True,
    alpha=0.8,
    label=f"With 3D ({len(data[cond]):,d})",
)

# Axes
ax.legend(loc="upper right")

fig.savefig(os.path.join("..", "gfx", "rms_shape.png"), facecolor="white", dpi=150)
fig.savefig(os.path.join("..", "gfx", "rms_shape.pgf"))

In [16]:
fig, ax = plt.subplots(
    figsize=fs.figsize(0.5), gridspec_kw={"right": 0.975, "top": 0.95, "bottom": 0.15}
)

# Plot FINK Sample
r = [0, 15]
b = 50


# Histogram FINK
ax.hist(
    data.loc[maskFINK, "SHG1G2_chi2red"],
    range=r,
    bins=b,
    color=color_fink,
    density=True,
    alpha=0.8,
    label=f"FINK ({len(data[maskFINK]):,d})",
)


# Histogram 3dShape
cond = maskFINK & data.sso_name.isin(damit.sso_name)
ax.hist(
    data.loc[cond, "SHG1G2_chi2red"],
    range=r,
    bins=b,
    color="slategray",
    density=True,
    alpha=0.8,
    label=f"With 3D ({len(data[cond]):,d})",
)

# Axes
ax.legend(loc="upper right")

fig.savefig(os.path.join("..", "gfx", "chi2_shape.png"), facecolor="white", dpi=150)
fig.savefig(os.path.join("..", "gfx", "chi2_shape.pgf"))

In [17]:
s = 3000
data[cond].sort_values("SHG1G2_chi2red").reset_index().loc[
    s : s + 10,
    ["ssnamenr", "SHG1G2_R", "SHG1G2_obliquity", "SHG1G2_chi2red", "SHG1G2_rms"],
]

,ssnamenr,SHG1G2_R,SHG1G2_obliquity,SHG1G2_chi2red,SHG1G2_rms
3000,28954,0.705206,65.652350,10.851988,0.336959
3001,2115,0.522816,8.763524,10.857574,0.110782
3002,15954,0.876663,128.797611,10.859661,0.211466
3003,16018,0.614215,74.456650,10.869347,0.177837
3004,65887,0.347097,165.027607,10.869720,0.256636
3005,1281,0.793595,112.695920,10.876826,0.116011
3006,9241,0.887491,77.028008,10.898327,0.183928
3007,2504,0.425269,170.677484,10.900497,0.104986
3008,4790,0.915747,151.790663,10.901915,0.150934
3009,4669,0.836119,42.481741,10.911373,0.144542


Some LC signature I think
- 9241


In [18]:
all = data[maskFINK].merge(damit, on="sso_name", how="left")

fig, ax = plt.subplots(
    figsize=fs.figsize(0.5), gridspec_kw={"right": 0.975, "top": 0.95, "bottom": 0.15}
)

ax.scatter(all.a / all.b, all.SHG1G2_rms, s=1, alpha=0.5)
ax.set_xlabel("a/b")
ax.set_ylabel("RMS")

fig.savefig(os.path.join("..", "gfx", "rms_vs_ab.png"), facecolor="white", dpi=150)


fig, ax = plt.subplots(
    figsize=fs.figsize(0.5), gridspec_kw={"right": 0.975, "top": 0.95, "bottom": 0.15}
)

ax.scatter(all.R, all.SHG1G2_rms, s=1, alpha=0.5)
ax.set_xlabel("R")
ax.set_ylabel("RMS")

fig.savefig(os.path.join("..", "gfx", "rms_vs_R.png"), facecolor="white", dpi=150)

# Test scatter DAMIT.R vs FINK.R

In [19]:
fig, ax = plt.subplots(
    figsize=fs.figsize(0.5), gridspec_kw={"right": 0.975, "top": 0.95, "bottom": 0.15}
)

# Plot FINK Sample
r = [0, 1]
b = 50

all = data[maskFINK].merge(damit, on="sso_name", how="left")

# Histogram
ax.scatter(
    all["SHG1G2_R"],
    all.R,
    s=1,
    marker=".",
    # color="slategray",
    # density=True,
    alpha=0.5,
    # label=f"DAMIT ({len(damit):,d})",
)


fig.savefig(
    os.path.join("..", "gfx", "oblateness-scatter.png"), facecolor="white", dpi=150
)
# fig.savefig(os.path.join("..","gfx","oblateness-scatter.pgf"))

# R vs obliquity and range lambda

In [57]:
fig, ax = plt.subplots(
    1,
    2,
    figsize=fs.figsize(0.5),
    gridspec_kw={
        "right": 0.98,
        "top": 0.95,
        "bottom": 0.15,
        "left": 0.1,
        "wspace": 0.02,
    },
    sharey=True,
)

# Histogram ranges and bins
r_R = [0.3, 1]
r_Lambda = [0, 90]
r_obliquity = [0, 90]

b_R = 35
b_Lambda = 36
b_obliquity = 36

vmin, vmax = 0, 150

# --------------------------------------------------------------------------------
# R vs Lambda
Lambda = np.degrees(
    np.arccos(data.loc[maskFINK, "SHG1G2_min_cos_lambda"])
    - np.arccos(data.loc[maskFINK, "SHG1G2_max_cos_lambda"])
)
_,_,_, im0 = ax[0].hist2d(
    Lambda,
    data.loc[maskFINK, "SHG1G2_R"],
    range=[r_Lambda, r_R],
    bins=[b_Lambda, b_R],
    cmap="Blues",
    norm="linear",
    rasterized=True,
    vmin=vmin, vmax=vmax,
    # label=f"DAMIT ({len(damit):,d})",
)


# --------------------------------------------------------------------------------
# Color bar
axinsr = inset_axes(
    ax[0],
    width="50%",  # width: 50% of parent_bbox width
    height="5%",  # height: 5%
    loc="lower right",
)
cbar_r = fig.colorbar(im0, cax=axinsr, orientation="horizontal")
cbar_r.ax.set_xticks([0,50,100,150])
cbar_r.ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
# cbar_r.ax.set_xticklabels(["1", "10", "100", "1000"], fontsize="small")



# --------------------------------------------------------------------------------
# R vs obliquity
obli = data.loc[maskFINK, "SHG1G2_obliquity"]
obli[obli > 90] = 180 - obli[obli > 90]

ax[1].hist2d(
    obli,
    data.loc[maskFINK, "SHG1G2_R"],
    range=[r_obliquity, r_R],
    bins=[b_obliquity, b_R],
    cmap="Blues",
    norm="linear",
    rasterized=True,
    # vmin=vmin, vmax=vmax,

)


# --------------------------------------------------------------------------------
# Axes
ax[0].set_ylabel("Oblateness R")
ax[0].set_xlabel(r"$\Delta \Lambda$ ($^{\circ}$)")
ax[1].set_xlabel(r"Obliquity ($^{\circ}$)")

ax[0].set_xticks(range(0, 91, 30))
ax[1].set_xticks(range(0, 91, 30))


# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join("..", "gfx", "article", "oblateness-issues.png"),
    facecolor="white",
    dpi=150,
)
fig.savefig(os.path.join("..", "gfx", "article", "oblateness-issues.pgf"))

In [60]:
cols = [
    "sso_number",
    "sso_name",
    "sso_class",
    "orbital_elements.semi_major_axis.value",
    "orbital_elements.eccentricity.value",
    "orbital_elements.inclination.value",
    "orbital_elements.node_longitude.value",
    "orbital_elements.perihelion_argument.value",
    "orbital_elements.mean_anomaly.value",
    "orbital_elements.mean_motion.value",
    "family.family_number",
    "family.family_name",
    "proper_elements.proper_semi_major_axis.value",
    "proper_elements.proper_eccentricity.value",
    "proper_elements.proper_inclination.value",
    "proper_elements.proper_sine_inclination.value",
    "tisserand_parameters.Jupiter.value",
    "albedo.value",
    "absolute_magnitude.value",
    "diameter.value",
    "taxonomy.class",
    "taxonomy.complex",
    "taxonomy.waverange",
    "taxonomy.scheme",
    "taxonomy.technique",
    "colors.g-r.color.value",
    "colors.g-r.color.error.min",
    "colors.g-r.color.error.max",
    "colors.g-r.facility",
    "colors.g-r.observer",
    "colors.g-r.epoch",
    "colors.g-r.delta_time",
    "colors.g-r.id_filter_1",
    "colors.g-r.id_filter_2",
    "colors.g-r.phot_sys",
    "colors.g-r.technique",
    "spins.1.obliquity",
    "spins.1.RA0.value",
    "spins.1.DEC0.value",
    "spins.1.RA0.error.max",
    "spins.1.DEC0.error.max",
    "spins.1.long.value",
    "spins.1.lat.value",
    "spins.1.technique",
    "spins.2.obliquity",
    "spins.2.RA0.value",
    "spins.2.DEC0.value",
    "spins.2.RA0.error.max",
    "spins.2.DEC0.error.max",
    "spins.2.long.value",
    "spins.2.lat.value",
    "spins.2.technique",
    "spins.3.obliquity",
    "spins.3.RA0.value",
    "spins.3.DEC0.value",
    "spins.3.RA0.error.max",
    "spins.3.DEC0.error.max",
    "spins.3.long.value",
    "spins.3.lat.value",
    "spins.3.technique",
    "spins.4.obliquity",
    "spins.4.RA0.value",
    "spins.4.DEC0.value",
    "spins.4.RA0.error.max",
    "spins.4.DEC0.error.max",
    "spins.4.long.value",
    "spins.4.lat.value",
    "spins.4.technique",
]
bft = pd.read_parquet(bft_file, columns=cols)

In [62]:
all_bft = all.merge(bft, left_on="sso_name", right_on="sso_name", how="left")

In [70]:
fig, ax = plt.subplots()

vmin=0
vmax=500

r_H = [12,18]
b_H = 12

ax.hist2d( all_bft["SHG1G2_R"], all_bft["absolute_magnitude.value"],  rasterized=True,
          range=[r_R, r_H], bins=[b_R, b_H], cmap="Blues", norm="linear", vmin=vmin, vmax=vmax,)


# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join("..", "gfx", "test-R.png"),
    facecolor="white",
    dpi=180,
)